In [6]:
# ----- IMPORTS --
import json
from datetime import datetime
from src.sales_etl_functions import *
from src.utils.config import *
from src.utils.logger import *
import psycopg2

import warnings
warnings.filterwarnings('ignore')


In [7]:
# ----- CHECK DB CONNECTION --
db_connected = check_db_connection(DB_CONFIG)

Database connection successful!


In [8]:
# ----- LOAD SOURCE TABLES --

load_sales_to_raw(DATA_PATH, PROCESSED_DATA_DIR, DB_CONFIG)
df_raw = extract_sales_from_raw(DB_CONFIG)

In [ ]:
# ----- TRANSFORMATIONS AND WRITE TO TABLE -- 

if not df_raw.empty:
    transformed_records, invalid_records = transform_sales(df_raw)

    df_transformed = pd.DataFrame(transformed_records)

    load_sales_to_silver(df_transformed, DB_CONFIG)

    update_sales_etl_metadata(DB_CONFIG)

    safe_invalid_records = make_json_safe(invalid_records)

    with open(INVALID_SALES_LOG_PATH, "w") as f:
        json.dump(safe_invalid_records, f, indent=4)
    
    logger.info(f"Saved {len(safe_invalid_records)} invalid records to {INVALID_SALES_LOG_PATH}")
else:
    logger.info("No new RAW data found. Pipeline finished.")